In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta, date
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
# Load in price data
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

In [ ]:
price_data.id = price_data.id.astype('str')

In [ ]:
miami_example = price_data[price_data['id'] == '52364652.0']

In [ ]:
start_time = time.time()
miami_example['pull_time'] = pd.to_datetime(miami_example['pull_time'])
print('done with datatype change')
middle_time = time.time()
miami_example['pull_time'] = miami_example['pull_time'].dt.date
print('done with change to date')
end_time = time.time()

print('Start Time '+ str(start_time))
print('Middle Time '+ str(middle_time-start_time))
print('End Time '+ str(end_time-start_time))


In [ ]:
miami_example_by_pull_time = miami_example.groupby(['id','pull_time'])['cleaning_fee','service_fee','total_price'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pull_time','check_in'])['cleaning_fee','service_fee','total_price'].count().reset_index()

# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['service_fee'],
                    mode='lines',
                    name='Service Fee'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pull_time'], y=miami_example_by_pull_time['total_price'],
                    mode='lines', 
                    name='Total Price'))
miami_example_grouped_fig.update_layout(title_text="Pricing Data by Date", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/price_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(miami_example_by_pull_time_count, x="pull_time",nbins=50)
fig.show()

In [ ]:
start_time = time.time()
price_data['pull_time'] = pd.to_datetime(price_data['pull_time'])
print('done with datatype change')
middle_time = time.time()
price_data['pull_time'] = price_data['pull_time'].dt.date ## Do we actually need this step if we structure it like the occupancy data? (greater than/less than pull on dates)
print('done with change to date')

end_time = time.time()
print('Start Time '+ str(start_time))
print('Middle Time '+ str(middle_time-start_time))
print('End Time '+ str(end_time-start_time))



In [ ]:
days_to_subtract = 30
last_day_from = pd.to_datetime('05/31/2022')
date_30_days_ago = last_day_from - timedelta(days=days_to_subtract)
pull_time_old = pd.to_datetime('04/20/2022')
pull_time_current = pd.to_datetime('05/05/2022')

price_data_last30 = price_data[(price_data['check_in'] > str(date_30_days_ago)) & (price_data['check_in'] <= str(last_day_from))]
price_data_last30['check_in'] = pd.to_datetime(price_data_last30['check_in'])
price_data_last30['day_of_week'] = price_data_last30['check_in'].dt.dayofweek
print(price_data_last30.shape)
price_data_previous_pull = price_data_last30[price_data_last30['pull_time'] == pull_time_old]
print(price_data_previous_pull.shape)
price_data_current_pull = price_data_last30[price_data_last30['pull_time'] == pull_time_current]
print(price_data_current_pull.shape)
price_data_previous_pull_avgs = price_data_previous_pull.groupby(['id','day_of_week']).mean().reset_index()
print(price_data_previous_pull_avgs.shape)
price_data_current_pull_avgs = price_data_current_pull.groupby(['id','day_of_week']).mean().reset_index()
print(price_data_current_pull_avgs.shape)
price_trend = price_data_previous_pull_avgs.merge(price_data_current_pull_avgs, how='inner', on='id')
print(price_trend.shape)
price_trend.to_csv('price_trend_miami_for_table.csv')

In [ ]:
price_data_last30['pull_time'].unique()

In [ ]:
# Do a delta from month of June price vs the month of May price for all properties to see increase vs decrease Month over Month
# Then, merge each id with its zipcode, state, and city (if city is there) from the flat csv files used in the eda_summary (listings_w_zips.csv)
# Then, groupby on zipcode with mean on delta from June-May to get average price increase vs decrease in that state/zip/city
"""
price_data_last30_avgs = price_data_last30.groupby(['id','pull_time']).mean().reset_index()
price_data_last30_avgs.to_csv('price_data_last30_avgs.csv')
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_data_last30_avgs.id = price_data_last30_avgs.id.astype('float64')
price_data_last30_avgs_w_loc = price_data_last30_avgs.merge(data_w_listing_loc, how='inner', on='id')
florida_price_trends = price_data_last30_avgs_w_loc[price_data_last30_avgs_w_loc['State'] == 'FL']
florida_price_trends = florida_price_trends.groupby('pull_time').mean(['cleaning_fee','service_fee','total_price']).reset_index()

#price_data_date = price_data.groupby(['check_in'])['cleaning_fee','service_fee','total_price'].apply(lambda row: np.sum(row)/len(row)).reset_index()

# Create traces
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""

In [ ]:
price_trend['cleaning_fee_delta'] = price_trend['cleaning_fee_y'] - price_trend['cleaning_fee_x']
price_trend['service_fee_delta'] = price_trend['service_fee_y'] - price_trend['service_fee_x']
price_trend['total_price_delta'] = price_trend['total_price_y'] - price_trend['total_price_x']
price_trend['cleaning_fee_delta_pct'] = price_trend['cleaning_fee_delta']/price_trend['cleaning_fee_x']*100
price_trend['service_fee_delta_pct'] = price_trend['service_fee_delta']/price_trend['service_fee_x']*100
price_trend['total_price_delta_pct'] = price_trend['total_price_delta']/price_trend['total_price_x']*100
price_trend['display_price_delta_pct'] = price_trend['total_price_delta_pct'] - price_trend['service_fee_delta_pct'] - price_trend['cleaning_fee_delta_pct']
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_trend['id'] = price_trend['id'].astype('float64')
price_trend_wloc = price_trend.merge(data_w_listing_loc, how='inner', on='id')

In [ ]:
florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']
florida_price_trends_zip = florida_price_trends.groupby('zipcode').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()
florida_price_trends_city = florida_price_trends.groupby('City').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()

import plotly.graph_objects as go

zips = florida_price_trends_zip['zipcode']
vals = [florida_price_trends_zip['display_price_delta_pct'], florida_price_trends_zip['cleaning_fee_delta_pct'], florida_price_trends_zip['service_fee_delta_pct'], florida_price_trends_zip['total_price_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Display Price</b>'], ['<b>Cleaning Fee</b>'], ['<b>Service Fee</b>'],['<b>Total Price</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Pricing Trends in Miami (%) (M/M)',
                   title_x=0.5, showlegend=False)
florda_price_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_price_trends_table_june19.png"
florda_price_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_price_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [ ]:
# make a new notebook called occupancy_eda.ipynb and copy all this code except switch out occupancy and change up logic and table (maybe do like month over month occupancy or 3 month outlook on occupancy)
# (could be like occupancy rate and if its green it went up and if its red it went down)

In [ ]:
fwd_looking_days = 28
last_day_from = pd.to_datetime('05/31/2022')
pull_time_current = pd.to_datetime('05/05/2022') # Would be set as date for "today" in the future
fwd_looking_date_current = pull_time_current + timedelta(days=fwd_looking_days)
pull_time_old = last_day_from - timedelta(days=7)
fwd_looking_date_old = pull_time_old + timedelta(days=fwd_looking_days)

price_data_last30_current = price_data[(price_data['check_in'] > str(pull_time_current)) & (price_data['check_in'] <= str(fwd_looking_date_current))]
price_data_last30_old = price_data[(price_data['check_in'] > str(pull_time_old)) & (price_data['check_in'] <= str(fwd_looking_date_old))]
print(price_data_last30_current.shape)
print(price_data_last30_old.shape)
price_data_last30_current['check_in'] = pd.to_datetime(price_data_last30_current['check_in'])
price_data_last30_old['check_in'] = pd.to_datetime(price_data_last30_old['check_in'])
price_data_last30_current['day_of_week'] = price_data_last30_current['check_in'].dt.dayofweek
price_data_last30_old['day_of_week'] = price_data_last30_old['check_in'].dt.dayofweek
price_data_previous_pull_avgs = price_data_last30_current.groupby(['id','day_of_week']).mean().reset_index()
print(price_data_previous_pull_avgs.shape)
price_data_current_pull_avgs = price_data_last30_old.groupby(['id','day_of_week']).mean().reset_index()
print(price_data_current_pull_avgs.shape)
price_trend = price_data_previous_pull_avgs.merge(price_data_current_pull_avgs, how='inner', on='id')
print(price_trend.shape)
price_trend.to_csv('price_trend_miami_for_table_weekly.csv')

In [ ]:
price_trend['cleaning_fee_delta'] = price_trend['cleaning_fee_y'] - price_trend['cleaning_fee_x']
price_trend['service_fee_delta'] = price_trend['service_fee_y'] - price_trend['service_fee_x']
price_trend['total_price_delta'] = price_trend['total_price_y'] - price_trend['total_price_x']
price_trend['cleaning_fee_delta_pct'] = price_trend['cleaning_fee_delta']/price_trend['cleaning_fee_x']*100
price_trend['service_fee_delta_pct'] = price_trend['service_fee_delta']/price_trend['service_fee_x']*100
price_trend['total_price_delta_pct'] = price_trend['total_price_delta']/price_trend['total_price_x']*100
price_trend['display_price_delta_pct'] = price_trend['total_price_delta_pct'] - price_trend['service_fee_delta_pct'] - price_trend['cleaning_fee_delta_pct']
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_trend['id'] = price_trend['id'].astype('float64')
price_trend_wloc = price_trend.merge(data_w_listing_loc, how='inner', on='id')

In [ ]:
florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']
florida_price_trends_zip = florida_price_trends.groupby('zipcode').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()

import plotly.graph_objects as go

zips = florida_price_trends_zip['zipcode']
vals = [florida_price_trends_zip['display_price_delta_pct'], florida_price_trends_zip['cleaning_fee_delta_pct'], florida_price_trends_zip['service_fee_delta_pct'], florida_price_trends_zip['total_price_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Display Price</b>'], ['<b>Cleaning Fee</b>'], ['<b>Service Fee</b>'],['<b>Total Price</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Pricing Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_price_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_price_trends_table_weekly.png"
florda_price_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_price_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [ ]:
florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']
florida_price_trends_city = florida_price_trends.groupby('City').mean(['cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()

import plotly.graph_objects as go

cities = florida_price_trends_city['City']
vals = [florida_price_trends_city['display_price_delta_pct'], florida_price_trends_city['cleaning_fee_delta_pct'], florida_price_trends_city['service_fee_delta_pct'], florida_price_trends_city['total_price_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>City</b>'], ['<b>Display Price</b>'], ['<b>Cleaning Fee</b>'], ['<b>Service Fee</b>'],['<b>Total Price</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [cities, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Pricing Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_price_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_price_trends_table_cities_weekly.png"
florda_price_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_price_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [ ]:
# Group by day of week for price comparison
#price_trend = pd.read_csv('price_trend_miami_for_table.csv')

# Calculate pull_time deltas for price
price_trend['cleaning_fee_delta'] = price_trend['cleaning_fee_y'] - price_trend['cleaning_fee_x']
price_trend['service_fee_delta'] = price_trend['service_fee_y'] - price_trend['service_fee_x']
price_trend['total_price_delta'] = price_trend['total_price_y'] - price_trend['total_price_x']
#price_trend['display_price_delta'] = price_trend['total_price_delta'] - price_trend['service_fee_delta'] - price_trend['cleaning_fee_delta']
price_trend['cleaning_fee_delta_pct'] = price_trend['cleaning_fee_delta']/price_trend['cleaning_fee_x']*100
price_trend['service_fee_delta_pct'] = price_trend['service_fee_delta']/price_trend['service_fee_x']*100
price_trend['total_price_delta_pct'] = price_trend['total_price_delta']/price_trend['total_price_x']*100
#price_trend['display_price_delta_pct'] = price_trend['display_price_delta']/(price_trend['total_price_x']-price_trend['service_fee_x']-price_trend['cleaning_fee_x'])*100
price_trend['display_price_delta_pct'] = price_trend['total_price_delta_pct'] - price_trend['service_fee_delta_pct'] - price_trend['cleaning_fee_delta_pct']
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
price_trend['id'] = price_trend['id'].astype('float64')
price_trend_wloc = price_trend.merge(data_w_listing_loc, how='inner', on='id')

# Get Florida data for example viz
florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']

# Pivot table on zipcode and day of the week
florida_price_trends_zip_dow = florida_price_trends.pivot_table(index=['zipcode','day_of_week_x'], values='total_price_delta_pct', aggfunc='mean')



In [ ]:
florida_price_trends_zip_dow

In [ ]:
import plotly.express as px
from urllib.request import urlopen
import json
#with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
#    counties = json.load(response)

with urlopen('https://raw.githubusercontent.com/OpenDataDE/State-zip-code-GeoJSON/master/fl_florida_zip_codes_geo.min.json') as response:
    zipcodes = json.load(response)

florida_price_trends_zipformap = florida_price_trends.groupby('zipcode').mean('total_price_delta_pct').reset_index()

fig = px.choropleth(florida_price_trends_zipformap, 
                    geojson=zipcodes, 
                    locations='zipcode', 
                    color='total_price_delta_pct',
                    color_continuous_scale="RdYlGn",
                    featureidkey="properties.ZCTA5CE10",
                    range_color=(-100,100),
                    scope="usa"
                    #labels={'Price Change':'Cluster_Category'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
filename = "newsletter_features/florida_price_trends_heatmap_zip_weekly.png"
fig.write_image(filename, engine='kaleido', width=875, height=700)
fig.show()

In [ ]:
zipcodes[0][0]

In [ ]:
import plotly.graph_objects as go

#headers = florida_price_trends_zip_dow['zipcode']
#vals = florida_price_trends_zip_dow['total_price_delta_pct']
#font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_for_dow = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Monday</b>'],['<b>Tuesday</b>'],['<b>Wednesday</b>'],['<b>Thursday</b>'],['<b>Friday</b>'],['<b>Saturday</b>'],['<b>Sunday</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values=[florida_price_trends_zip_dow['0'],
                            florida_price_trends_zip_dow['1'],
                            florida_price_trends_zip_dow['2'],
                            florida_price_trends_zip_dow['3'],
                            florida_price_trends_zip_dow['4'],
                            florida_price_trends_zip_dow['5'],
                            florida_price_trends_zip_dow['6']],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              #font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Pricing Trends in Miami (%) (W/W) By Day of Week',
                   title_x=0.5, showlegend=False)
florda_price_trend_fig = go.Figure(data=[table_for_dow], layout=layout)
filename = "newsletter_features/florida_price_trends_table_dow.png"
florda_price_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_price_trend_fig.show()

In [ ]:
florida_price_trends_zip_dow_linechart = florida_price_trends_zip_dow.reset_index()
fig_line_prices_zips = px.line(florida_price_trends_zip_dow_linechart, x="day_of_week_x", y="total_price_delta_pct", color='zipcode',                  labels={
                     "day_of_week_x": "Day of Week",
                     "total_price_delta_pct": "Pct Price Change"
                 })
fig_line_prices_zips.update_layout(title='Price Changes in Miami W/W Monday-Sunday')
fig.update_xaxes(range=[-100, 100])
filename = "newsletter_features/florida_price_trends_lines_zip_weekly.png"
fig.write_image(filename, engine='kaleido', width=875, height=700)
fig_line_prices_zips.show()

In [ ]:
price_trend = pd.read_csv('price_trend_miami_for_table.csv')

data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','lat','lng','zipcode','City','Metro','CountyName','State']]
price_trend['id'] = price_trend['id'].astype('float64')
price_trend_wloc = price_trend.merge(data_w_listing_loc, how='inner', on='id')

price_trend_wloc.to_csv('price_trends_for_map.csv')

# Get Florida data for example viz
#florida_price_trends = price_trend_wloc[price_trend_wloc['State'] == 'FL']

# Pivot table on zipcode and day of the week
# Cant do because we dont have data in local csv and now we have new crazy amount of data in local file
## -> remove 80% of data in local
"""
price_trend_wloc_pvttbl = price_trend_wloc.pivot_table(index=['zipcode','day_of_week_x'], values='total_price_delta_pct', aggfunc='mean')

price_trend_wloc_pvttbl.to_csv('price_trend_wloc_pvttbl.csv')
"""



In [ ]:
price_trend_wloc.to_csv('price_trends_for_map.csv')


price_trend_wloc_pvttbl = price_trend_wloc.pivot_table(index=['zipcode','day_of_week_x'], values='total_price_delta_pct', aggfunc='mean')
price_trend_wloc_pvttbl.to_csv('price_trend_wloc_pvttbl.csv')


price_trends_wloc_zip_for_table = price_trend_wloc.groupby('zipcode').mean(['display_price_delta_pct','cleaning_fee_delta_pct','service_fee_delta_pct','total_price_delta_pct']).reset_index()
price_trends_wloc_zip_for_table.to_csv('price_trends_wloc_zip_for_table.csv')

In [ ]:
# Example for if we want to use Density plots in place of choropleth while working GeoJSON issue for zipcodes

import plotly.express as px
avg_lat = price_trend_wloc['lat'].mean()
avg_lng = price_trend_wloc['lng'].mean()
dens_map_price = px.density_mapbox(price_trend_wloc, lat='lat', lon='lng', z='total_price_delta_pct', radius=10,
                        center=dict(lat=avg_lat, lon=avg_lng), zoom=3, color_continuous_scale='RdYlGn', range_color=(-100,100),
                        mapbox_style="stamen-terrain")
dens_map_price.show()

In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
        .config("spark.sql.extension",
        "io/delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog",
            "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:736)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:271)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1105)
	at org.apache.hadoop.fs.FileUtil.chmod(FileUtil.java:1091)
	at org.apache.spark.util.Utils$.fetchFile(Utils.scala:571)
	at org.apache.spark.SparkContext.addFile(SparkContext.scala:1633)
	at org.apache.spark.SparkContext.$anonfun$new$13(SparkContext.scala:510)
	at org.apache.spark.SparkContext.$anonfun$new$13$adapted(SparkContext.scala:510)
	at scala.collection.immutable.List.foreach(List.scala:431)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:510)
	at org.apache.spark.api.java.JavaSparkC

In [4]:
from deltalake import DeltaTable

#people = spark.read.format('delta').load('s3://airbnb-scraper-bucket-0-1-1/data/beta_data_tables/occupancy_lag_metrics/')
dt = DeltaTable('s3://airbnb-scraper-bucket-0-1-1/data/beta_data_tables/occupancy_lag_metrics/')

PyDeltaTableError: Failed to load checkpoint: Failed to read checkpoint content: Failed to read S3 object content: Request ID: None Body: <?xml version="1.0" encoding="UTF-8"?>
<Error><Code>PermanentRedirect</Code><Message>The bucket you are attempting to access must be addressed using the specified endpoint. Please send all future requests to this endpoint.</Message><Endpoint>s3.amazonaws.com</Endpoint><Bucket>airbnb-scraper-bucket-0-1-1</Bucket><RequestId>Y7AHWRZKEJ0GB5FG</RequestId><HostId>J3sjxqQ34il/SWVEXK33NosYwCMDGaiuU7NmMemc9I4K6pHZdFw57LAkSqXTBOWv7FiYuJfboDU=</HostId></Error>